In [1]:
import faiss
index = faiss.read_index(r'D:\desktop\code\output\v2\FAISS\bge_large_v1.5\faiss_index\index.faiss')
print(index.ntotal)  

1163


In [2]:
import pickle

with open(r"D:\desktop\code\output\v1\FAISS\bge_large_v1.5\faiss_index\index.pkl", "rb") as f:
    index_data = pickle.load(f)
print(index_data) 


(<langchain_community.docstore.in_memory.InMemoryDocstore object at 0x0000028F9CCCA340>, {0: 'b75bab45-51db-4976-b2b0-29da5b3c41c5', 1: 'f3337bed-f4d5-4ef8-8282-9fe1fa7cd992', 2: '478e73d6-be6d-4787-9331-adcc7220ac97', 3: '47763eed-baf6-4b92-bf4f-219eeeba2e73', 4: 'fc1e8656-8b8b-4f3b-997a-c7efb83730ac', 5: '2508147d-e2b5-4dec-8f77-c6c8638e394c', 6: '7230e88e-a75f-4296-9652-ffb48cdacb6e', 7: 'f1b58c9f-14d4-45da-8ca7-bf6431f1d9a8', 8: '32ce1279-4ff7-4f6f-b975-a22ce4186019', 9: '6d8dce5b-7717-451c-be1d-12d5fd03c446', 10: 'a5736721-cebf-465e-bd4d-b2424bef943e', 11: '2b0817cd-b00c-48b7-905d-b4c97f5b6ab7', 12: '0b224674-9193-474a-887b-7ef5cfd56d48', 13: '000de9cc-9538-4175-8034-115fd1ddeb9c', 14: 'a182fe80-7c85-43ba-9cff-e9d69268cada', 15: 'e7f10d57-ac5f-4484-a2aa-f5d431a800f9', 16: '5c50dc7c-40e2-4cb6-adaf-d0d77c5a6032', 17: 'b84d9bef-e9f5-448a-9085-38985c8f6eb8', 18: '5837ac44-4300-4cdb-8f0b-c774b01048a6', 19: 'c02672b9-bffd-418c-9fb8-4eb6470377a9', 20: 'ce1d0519-ebeb-48ec-876d-83e19923c05

In [1]:
import json

json_file = r'D:\desktop\code\cache\semantic_merged_chunks.json'  # 替换成你的文件路径

with open(json_file, 'r', encoding='utf-8') as f:
    data = json.load(f)

num_chunks = len(data)
print(f"JSON 文件中共有 {num_chunks} 个文本块")


JSON 文件中共有 707 个文本块


In [2]:
import json
import requests
import time
from tqdm import tqdm
from difflib import get_close_matches
from opencc import OpenCC

API_KEY = "2a2299dd95944956b69397a89113d5a7.GW5jR7NYhANOuGES"  
API_URL = "https://open.bigmodel.cn/api/paas/v4/chat/completions"
HEADERS = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}"
}

# 创建OpenCC对象，设置从繁体到简体的转换
cc = OpenCC('t2s')

def build_prompt(question, answer):
    return f"""
你是一位保险领域的资深专家，请对以下问答对进行评分，满分为5分。

【问题】：{question}
【回答】：{answer}

请你从保险领域专业视角，根据uuid定位到QA对对应的文本块，然后进行比较并从以下几个方面评估给定的保险领域 QA 对的质量：

1.准确性：判断问题和答案是否与保险专业知识、相关条款及给定文档内容高度一致，杜绝事实性错误。若答案与权威保险资料、行业标准规范相悖，或者对问题的表述偏离文档核心意思，则视为不准确。
2.完整性：检查问题和答案是否全面覆盖关键要点。对于保险条款解读类问题，需涵盖条款核心内容、适用条件、限制范围等；理赔流程类问题，要包含从报案到赔付的主要步骤、所需材料等。若有关键环节缺失，则判定为不完整。
3.清晰度：评估问题和答案是否清晰易懂，逻辑结构是否合理。在解释保险责任判定时，推理过程应依据合理逻辑从事故情况推导到责任归属；阐述保险产品特点时，各特点之间逻辑关系需清晰，避免出现逻辑混乱或跳跃的情况。


请按以下格式回答：
分数：X.X
评语：......
"""

def score_qa_pair(question, answer):
    prompt = build_prompt(question, answer)
    data = {
        "model": "glm-4",
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.2
    }
    try:
        response = requests.post(API_URL, headers=HEADERS, json=data)
        if response.status_code == 200:
            result = response.json()
            return result["choices"][0]["message"]["content"]
        else:
            return f"请求失败: {response.status_code} - {response.text}"
    except Exception as e:
        return f"异常: {str(e)}"

def find_best_matching_block(question, text_blocks, threshold=0.15):
    # 将问题转换为简体字
    question = cc.convert(question)
    # 提取所有文本块的文本内容，并转换为简体字
    block_texts = [cc.convert(block["content"]) for block in text_blocks]
    # 使用difflib找到最匹配的文本
    matches = get_close_matches(question, block_texts, n=1, cutoff=threshold)
    
    if matches:
        best_match = matches[0]
        # 找到匹配文本对应的索引
        index = block_texts.index(best_match)
        return text_blocks[index]["uuid"]
    else :
        # 无匹配时打印原因
        print(f"未找到匹配的文本块，问题: {question[:80]}...")
        print(f"可能原因：问题与文本块的相似度低于阈值 {threshold}")
        return "no_match"

def process_testset(qa_file, text_blocks_file, output_file):
    # 加载QA对文件
    with open(qa_file, "r", encoding="utf-8") as f:
        qa_list = json.load(f)
    
    # 加载文本块文件
    with open(text_blocks_file, "r", encoding="utf-8") as f:
        text_blocks = json.load(f)

    results = []
    
    print("\n开始处理QA对评分和映射...\n")
    
    for item in tqdm(qa_list, desc="处理进度", unit="对"):
        question = item.get("question", "")
        answer = item.get("answer", "")
        
        # 获取评分
        feedback = score_qa_pair(question, answer)
        item["score_feedback"] = feedback
        
        # 找到最匹配的文本块
        best_block_uuid = find_best_matching_block(question, text_blocks)
        
        # 记录结果
        results.append(item)
        
        # 输出当前QA对的评分和映射关系
        print("\n" + "="*50)
        print(f"问题: {question[:80]}...")
        print(f"答案: {answer[:40]}...")
        print(f"评分结果: {feedback[:60]}...")
        print(f"对应文本块UUID: {best_block_uuid}")
        print("="*50 + "\n")
        
        time.sleep(2)  # 控制API请求频率

    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)
    
    print("\n所有问答评分和映射已完成，结果保存在：", output_file)

# 调用函数，替换为你的实际文件路径
process_testset(
    r"D:\desktop\code\QA\test.json",      # QA对JSON文件路径
    r"D:\desktop\code\cache\all_docs_split_400_40.json",   # 文本块JSON文件路径
    "qa_scored_with_blocks111.json"           # 输出文件路径
)


开始处理QA对评分和映射...



处理进度:   0%|          | 0/97 [00:00<?, ?对/s]


问题: 什么是宏利环球货币保障计划？...
答案: 这是一款分红型终身人寿保险计划，提供身故赔偿、意外身故赔偿及现金价值积累功 能，...
评分结果: 分数：4.5

评语：这个QA对在保险领域的专业视角下，整体表现良好。在准确性方面，回答对“宏利环球货币保障计划”的描述...
对应文本块UUID: 9e2dcf4b-3d94-4052-b0e4-cbb588795da5



处理进度:   1%|          | 1/97 [00:07<12:17,  7.68s/对]


问题: 保单的身故赔偿如何计算？...
答案: 身故赔偿为以下两者的较高者：(a) 表格所示现金价值 + 终期红利（如有）；或 ...
评分结果: 分数：4.8

评语：这个QA对在保险领域的专业准确性、完整性和清晰度方面做得非常好。首先，在准确性方面，回答正确地描述...
对应文本块UUID: 6627fd53-2433-4ca3-9b66-6d40d73f9fc1



...


问题: 若未按时缴付保费，保单会失效吗？...
答案: 31天宽限期后仍未缴费，若保单有足够现金价值，将启动自动贷款代缴保费；否则保 单...
评分结果: 分数：4.7
评语：这个QA对在保险领域的专业准确性、完整性和清晰度方面做得相当好。在准确性方面，回答正确地描述了保单在...
对应文本块UUID: de639342-cf8c-4241-bc16-b781861f13aa



处理进度:  99%|█████████▉| 96/97 [14:25<00:09,  9.76s/对]


问题: 人民币保单能否自由兑换其他货币？...
答案: 可以，但人民币受外汇管制限制，兑换需符合当时政策，且汇率由宏利厘定，可能存 在波...
评分结果: 分数：4.2

评语：这个问答对在保险领域的专业视角下，表现出了较高的准确性，对人民币保单的兑换问题作出了符合实际的外汇...
对应文本块UUID: no_match



处理进度: 100%|██████████| 97/97 [14:34<00:00,  9.01s/对]


所有问答评分和映射已完成，结果保存在： qa_scored_with_blocks111.json
